In [ ]:
!pip install transformers
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.3 MB/s eta 0:00:00


In [ ]:

# Importiamo le librerie necessarie
from transformers import AutoTokenizer, EncoderDecoderModel
import torch

# BERT Encoder-Decoder




In [ ]:

# Carichiamo in memoria un modello di tipo Encoder-Decoder basato su BERT e il suo relativo Tokenizzatore
model = EncoderDecoderModel.from_pretrained("patrickvonplaten/bert2bert_cnn_daily_mail")
tokenizer = AutoTokenizer.from_pretrained("patrickvonplaten/bert2bert_cnn_daily_mail")

config.json:   0%|          | 0.00/3.66k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

In [ ]:
model.eval()
model

In [ ]:
# Proviamo a fare inferenza e generare un riassunto di questo articolo:
ARTICLE_TO_SUMMARIZE = (
    "PG&E stated it scheduled the blackouts in response to forecasts for high winds "
    "amid dry conditions. The aim is to reduce the risk of wildfires. Nearly 800 thousand customers were "
    "scheduled to be affected by the shutoffs which were expected to last through at least midday tomorrow."
)
input_ids = tokenizer(ARTICLE_TO_SUMMARIZE, return_tensors="pt").input_ids

In [ ]:
# Generiamo il riassunto un maniera auto-regressiva
generated_ids = model.generate(input_ids)
generated_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(generated_text)


nearly 800 thousand customers were affected by the shutoffs. the aim is to reduce the risk of wildfires. nearly 800, 000 customers were expected to be affected by high winds amid dry conditions. pg & e said it scheduled the blackouts to last through at least midday tomorrow.


Vediamo ora il modello T5, sempre con architettura Encoder-Decoder

# T5

In [ ]:
# Importiamo il necessario per caricare in memoria il modello T5
from transformers import TFT5ForConditionalGeneration, T5Tokenizer

In [ ]:
# Carichiamo in memoria il modello e il relativo Tokenizzatore
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-small")
model = TFT5ForConditionalGeneration.from_pretrained("google/flan-t5-small", pad_token_id=tokenizer.eos_token_id)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
model.eval()
model

In [ ]:
# Generiamo il testo.
# Il modello T5 è stato allenato in modo tale da seguire delle brevi istruzioni poste all'inizio del testo di input
# Proviamo a continuare la generazione del testo

input_ids = tokenizer.encode('Continue the sentence: I was in Paris and decided that', return_tensors='pt')
greedy_output = model.generate(input_ids, max_length=50)
print("Output:\n" + 100 * '-')

print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
Paris was a good place to stay.


In [ ]:
# Proviamo questa volta invece a scrivere un riassunto del testo

input_ids = tokenizer.encode('summarize: PG&E stated it scheduled the blackouts in response to forecasts for high winds amid dry conditions.\
 The aim is to reduce the risk of wildfires. Nearly 800 thousand customers werescheduled to be affected by \
 the shutoffs which were expected to last through at least midday tomorrow.', return_tensors='pt')
greedy_output = model.generate(input_ids, max_length=50)
print("Output:\n" + 100 * '-')

print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
PG&E has cancelled a number of blackouts in response to forecasts for high winds.


In [ ]:
Importiamo e utilizziamo ora il modello GPT2, con la stessa architettura solo Decoder dei successivi modelli GPT-3, come abbiamo visto

# GPT2

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [ ]:
text = "Welcome to the open data science conference it is"
indexed_tokens = tokenizer.encode(text)
indexed_tokens

[14618, 284, 262, 1280, 1366, 3783, 4495, 340, 318]

In [ ]:
tokens_tensor = torch.tensor([indexed_tokens])
tokens_tensor

tensor([[14618,   284,   262,  1280,  1366,  3783,  4495,   340,   318]])

In [ ]:
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.eval()
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
with torch.no_grad():
    outputs = model(tokens_tensor)
    predictions = outputs[0]

In [ ]:
predicted_index = torch.argmax(predictions[0, -1, :]).item()
predicted_text = tokenizer.decode(indexed_tokens + [predicted_index])
predicted_text

'Welcome to the open data science conference it is a'

In [ ]:
start = 'Natural Language Processing is slowly becoming'
indexed_tokens = tokenizer.encode(start)

for i in range(75):
  tokens_tensor = torch.tensor([indexed_tokens])
  with torch.no_grad():
    outputs = model(tokens_tensor)
    predictions = outputs[0]
    predicted_index = torch.argmax(predictions[0, -1, :]).item()
    indexed_tokens = indexed_tokens + [predicted_index]

In [ ]:
predicted_text = tokenizer.decode(indexed_tokens)
print(predicted_text)

Natural Language Processing is slowly becoming a reality.

The first step is to create a language processing system that can be used to create a language. This is done by using a language processing system that is built on top of a language processing system.

The language processing system is a set of tools that can be used to create a language. The language processing system is a set of tools that can
